<a href="https://colab.research.google.com/github/PaulinaTarkowsk/sql_recap/blob/main/hyperparametrs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Wyszukiwanie siatkowe**

# **Głównymi argumentami konstruktora GridSearchCV są:**

**estimator** - jest to obiekt modelu, który zostanie dopasowany do danych. Może to być dowolny model z biblioteki scikit-learn, który posiada metody fit() i predict().

**param_grid** - jest to słownik lub lista słowników, w którym każdy klucz to nazwa hiperparametru, a wartość to lista wartości, które chcemy przetestować. GridSearchCV przetestuje wszystkie możliwe kombinacje tych wartości.

**scoring** - określa miarę jakości, która będzie używana do oceny modelu dla każdej kombinacji hiperparametrów. Może to być jedna z wbudowanych miar, takich jak accuracy czy mean_squared_error, lub niestandardowa funkcja oceny. Lista gotowych do użycia metryk, które możemy przekazać do tego argumentu, znajduje się tu.

**cv** - określa schemat podziału danych do walidacji krzyżowej. Może to być liczba podziałów (np. 5 dla walidacji krzyżowej 5-krotnej) lub obiekt generatora walidacji krzyżowej.

**n_jobs** - określa liczbę procesów, które mają być użyte do przeprowadzenia obliczeń w trakcie przeszukiwania siatki. Wartość -1 oznacza, że zostaną wykorzystane wszystkie dostępne procesory.


# **Klasa GridSearchCV udostępnia również kilka przydatnych metod, takich jak:**

**fit(X, y):** Metoda ta dopasowuje model do danych treningowych, przeprowadzając przeszukiwanie siatki hiperparametrów i oceniając model dla każdej kombinacji. Po zakończeniu dostępne są atrybuty, takie jak **best_params_, best_score_ i best_estimator_**, które zawierają odpowiednio najlepsze znalezione wartości hiperparametrów, wynik i najlepszy dopasowany model.

**predict(X):** Metoda ta dokonuje predykcji na podstawie danych X za pomocą najlepszego znalezionego modelu.

**score(X, y):** Metoda ta oblicza wynik modelu na podstawie danych X i etykiet y za pomocą domyślnej miary jakości.


In [ ]:
#Przykład użycia GridSearchCV dla algorytmu drzewa decyzyjnego:

import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Define parameters grid to find the best set
params_grid = {
    'max_depth': np.arange(3, 21, 2), # list of integers from 3 to 19 with step = 2
    'criterion': ["entropy", "gini"],
    'max_features': [None, "log2", "sqrt"],
}

# Create a model
model = DecisionTreeClassifier()

# Create GridSearchCV object
grid_search = GridSearchCV(model, params_grid, cv=5, scoring="recall")

# Fit training data using grid_search object
grid_search.fit(X_train, y_train)

# Best hyperparameters set found during grid search
best_params = grid_search.best_params_


#**Wyszukiwanie losowe**

#**Głównymi argumentami konstruktora klasy RandomizedSearchCV są:**

**estimator** — to obiekt modelu, który zostanie dopasowany do danych. Może to być dowolny model z biblioteki scikit-learn, który posiada metody **.fit() i .predict()**.

**param_distributions** — to słownik, w którym każdy klucz to nazwa hiperparametru, a wartość to rozkład prawdopodobieństwa lub lista wartości, które chcemy przetestować. RandomizedSearchCV losowo próbuje różne kombinacje wartości hiperparametrów z zadanego zakresu.

**n_iter** — określa liczbę iteracji, czyli liczbę prób losowych kombinacji wartości hiperparametrów do przetestowania.

**scoring** — określa miarę jakości, która będzie używana do oceny modelu dla każdej kombinacji hiperparametrów. Może to być jedna z wbudowanych miar, takich jak accuracy czy mean_squared_error, lub niestandardowa funkcja oceny.

**cv** — określa schemat podziału danych do walidacji krzyżowej. Może to być liczba podziałów (np. 5 dla walidacji krzyżowej 5-krotnej) lub obiekt generatora walidacji krzyżowej.

**n_jobs** — określa liczbę procesów, które mają być użyte do przeprowadzenia obliczeń w trakcie przeszukiwania losowego. Wartość -1 oznacza, że zostaną wykorzystane wszystkie dostępne procesory.

#**Klasa RandomizedSearchCV ma zaimplementowane analogiczne metody jak GridSearchCV**


In [3]:
#Przykład użycia RandomizedSearchCV dla maszyny wektorów nośnych:

from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Define hyperparameters params dict
param_dist = {
    'C': uniform(loc=0, scale=4),
    'gamma': uniform(loc=0, scale=0.1),
}

# Create a model instance
model = SVC()

# Create RandomizedSearchCV object
random_search = RandomizedSearchCV(model, param_dist, n_iter=10, cv=5)

# Fit model to training data
random_search.fit(X_train, y_train)

# The best set of hyperparameters found
best_params = random_search.best_params_


#**Wyszukiwanie bayesowskie**

In [ ]:
# Import all needed methods from hyperopt
from hyperopt import hp, fmin, tpe, Trials, space_eval

# Define hyperparameter space
space = {
    'min_samples_split': hp.uniform('min_samples_split', 5, 15),
    'min_samples_leaf': hp.choice('min_samples_leaf', [20, 30, 40]),
    'max_depth': hp.choice('max_depth', [None, 5, 10]),
    'criterion': hp.choice('criterion', ['gini', 'entropy'])
}

# Define function that measure chosen metric for given hyperparameter combination

def objective(params):
    # Create a model. Here: decision tree classifier
    model = DecisionTreeClassifier(params)

    # Fit model using training data
    model.fit(X_train, y_train)

    # Calculate chosen metric
    score = model.score(X_val, y_val) # val is a shortcut from validation

    # Return score with "-". As we want to maximize our metric,
    # but bayesian optimization goal is to minimize function, we need add minus sign
    return -score

# Initializer Trials object due to store all iterations and use them to further optimizations
trials = Trials()

# Call optimization function
best = fmin(
    fn=objective,
    space=space,
    max_evals=50,
    trails=trials
)

# Get best hyperparameters set found
best_params = space_eval(space, best)


**W powyższym przykładzie korzystaliśmy z metod, których wcześniej nie znaliśmy. Poniżej wyjaśnienie, za co odpowiedzialne są poszczególne fragmenty kodu:**

**space** — definiuje przestrzeń poszukiwań hiperparametrów, gdzie dla każdego hiperparametru określamy rodzaj rozkładu (hp.loguniform, hp.choice itp.) oraz zakres wartości.

**hp.choice(label, options)** — definiuje rozkład dyskretny, gdzie options to lista możliwych wartości dla hiperparametru,

**hp.uniform(label, low, high)** — definiuje rozkład jednostajny, gdzie wartości są generowane równomiernie między low a high,

**hp.randint(label, upper)** — definiuje rozkład całkowity, gdzie wartości są generowane jako liczby całkowite z zakresu od 0 do upper.

**hp.normal(label, mu, sigma)** — definiuje rozkład normalny o zadanych wartościach średniej (mu) i odchylenia standardowego (sigma).

**objective** — funkcja oceny, która przyjmuje jako argument zestaw hiperparametrów i ocenia wynik modelu dla danej kombinacji.

**trials** — to obiekt Trials, który śledzi postęp optymalizacji i przechowuje wyniki dla każdej iteracji.

**fmin** — to główna funkcja biblioteki hyperopt, która przeprowadza proces optymalizacji. Przyjmuje argumenty takie jak:

**fn** — funkcja oceny,

**space** — przestrzeń poszukiwań hiperparametrów,

**max_evals** — maksymalna liczba iteracji do wykonania,

**trials** — obiekt Trials.